In [ ]:
#| default_exp fastmcp_tools
#| test: false

"Shared FastMCP SSE server exposing parking & route tools."


In [ ]:
#| export
#| eval: false

from fastapi import FastAPI
from mcp.server.fastmcp import FastMCP


#   Tools implemented in 02_parking_tools.ipynb
from DataTalks.parking_tools import facility_prediction, parking_status

# ── Build the MCP server ───────────────────────────────────────────────────
mcp = FastMCP(port=9001)
sse_app = mcp.sse_app()               # ← Starlette app (SSE only)

# ── Wrap in FastAPI so we can add docs / health ───────────────────────────
app = FastAPI(
    title="FastMCP Tools",
    version="0.1.0",
    description="SSE gateway exposing parking & routing tools."
)
app.mount("/sse", sse_app)            # → http://tools:9001/sse


In [ ]:
#| export
#| eval: false

@app.get("/health", tags=["meta"])
async def health():
    "Simple liveness probe."
    return {"status": "ok"}


In [ ]:
#| export
#| eval: false
"Expose every routing helper from api_clients as an MCP tool."

from DataTalks.api_clients import (
    Point,
    osrm_car,
    osrm_walk,
    ors_bike,
    digitransit_pt,
    geocode, Route,  # handy for front-end address lookup
)

# ── OSRM driving ──────────────────────────────────────────────────────────
@mcp.tool()
async def car_route(
    src_lat: float, src_lon: float,
    dst_lat: float, dst_lon: float,
) -> Route:
    "Car route via public OSRM demo."
    return await osrm_car(Point(lat=src_lat, lon=src_lon),
                          Point(lat=dst_lat, lon=dst_lon))

# ── OSRM walking ───────────────────────────────────────────────────────────
@mcp.tool()
async def walk_route(
    src_lat: float, src_lon: float,
    dst_lat: float, dst_lon: float,
) -> Route:
    "Walking route via OSRM."
    return await osrm_walk(Point(lat=src_lat, lon=src_lon),
                           Point(lat=dst_lat, lon=dst_lon))

# ── OpenRouteService cycling ───────────────────────────────────────────────
@mcp.tool()
async def bike_route(
    src_lat: float, src_lon: float,
    dst_lat: float, dst_lon: float,
    profile: str = "cycling-regular",
) -> Route:
    "Cycling route via OpenRouteService (requires ORS_KEY env-var)."
    return await ors_bike(Point(lat=src_lat, lon=src_lon),
                          Point(lat=dst_lat, lon=dst_lon),
                          profile=profile)

# ── Digitransit public transport ───────────────────────────────────────────
@mcp.tool()
async def pt_route(
    src_lat: float, src_lon: float,
    dst_lat: float, dst_lon: float,
    dataset: str = "hsl",
) -> Route:
    "Fastest public-transport itinerary via Digitransit Routing v2."
    return await digitransit_pt(Point(lat=src_lat, lon=src_lon),
                                Point(lat=dst_lat, lon=dst_lon),
                                dataset=dataset)

# ── Nominatim geocoding (utility) ──────────────────────────────────────────
@mcp.tool()
async def geocode_osm(query: str) -> dict:
    "First OpenStreetMap location match for *query*."
    return (await geocode(query)).model_dump()


In [ ]:
#| export
#| eval: false
"Expose parking helpers from parking_tools as FastMCP tools."

from DataTalks.parking_tools import (
    nearest_parking_ids,
    parking_status,
    facility_prediction,
)

# ── MCP tool: IDs of nearest P+R facilities ───────────────────────────────
@mcp.tool()
async def nearest_facilities(
    lat: float, lon: float, radius_m: int = 800
) -> dict[str, int]:
    """
    Return `{capacityType: facility_id}` for the closest facilities
    within *radius_m* metres of (*lat*, *lon*).
    """
    return await nearest_parking_ids(lat, lon, radius_m=radius_m)

# ── MCP tool: live vacancy for one facility ───────────────────────────────
@mcp.tool()
async def live_vacancy(
    facility_id: int = 285,                  # Urheilopuisto default
    capacity_type: str = "CAR",
    usage: str = "PARK_AND_RIDE",
) -> dict:
    """
    Live parking status for a single facility.
    Returns JSON with keys: capacity, spacesAvailable, timestamp_utc, openNow.
    """
    return await parking_status(
        facility_id,
        capacity_type=capacity_type,
        usage=usage,
    )

# ── MCP tool: vacancy forecast (Fintraffic) ───────────────────────────────
@mcp.tool()
async def vacancy_forecast(
    facility_id: int = 285,
    hours: int = 24,
) -> list[dict]:
    """
    Fintraffic capacity forecast for *facility_id* over the next *hours*.
    """
    return await facility_prediction(facility_id, hours=hours)
